### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import re
import datetime 

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Semi-Monthly Closed Invoice\2022 Semi-Monthly Closed Invoice'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### ACCESSORIAL Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
        
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'ACCESSORIAL']
        
    
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    
    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

In [ ]:
df_list[len(df_list) - 1].sort_values('Accounting_CustomerID')

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

In [ ]:
df_output

### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [ ]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

In [ ]:
df_count.sort_values(['Count'])

In [ ]:
# month = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

In [ ]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

In [ ]:
df_count.to_excel("03_2022 Semi-Monthly Report Accessorial Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [ ]:
# month = np.array(['Jan', 'Feb'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [ ]:
df_output

In [ ]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False).head(12)

In [ ]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

In [ ]:
df_output.to_excel("03_2022 Semi-Monthly Report Accessorial.xlsx", index = False, header = True)

In [ ]:
# writer = pd.ExcelWriter('2022 Semi-Monthly Report Accessorial.xlsx', engine = 'xlsxwriter')

In [ ]:
# df_count.to_excel(writer, index = False, header = True, sheet_name = 'Summary')

In [ ]:
# df_output.to_excel(writer, index = False, header = True, sheet_name = 'Detail 01-02')

In [ ]:
# writer.save()

# MOVE FILES TO OUTPUT!

### HANDLING Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
    
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'HANDLING']
    
    
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    


    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [ ]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

In [ ]:
month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

In [ ]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

In [ ]:
df_count.to_excel("03_2022 Semi-Monthly Report Handling Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [ ]:
# month = np.array(['Jan'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [ ]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False)
df_output

In [ ]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

In [ ]:
df_output.to_excel("03_2022 Semi-Monthly Report Handling.xlsx", index = False, header = True)

In [ ]:
# writer1 = pd.ExcelWriter('2022 Semi-Monthly Report Handling.xlsx', engine = 'xlsxwriter')
# df_count.to_excel(writer1, index = False, header = True, sheet_name = 'Summary')
# df_output.to_excel(writer1, index = False, header = True, sheet_name = 'Detail 01-02')
# writer1.save()

# MOVE FILES TO OUTPUT!

### Others Generate Semi-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category']]
        
        
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] != 'STORAGE INCOME']
    
    df_select_detail['Bill Pay Category'] = np.where((df_select_detail['Bill Pay Category'] != 'HANDLING') & 
                                                     (df_select_detail['Bill Pay Category'] != 'ACCESSORIAL'), 
                                           'OTHERS', df_select_detail['Bill Pay Category'])
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'OTHERS']
        
 

    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    
 

    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

### Generate Semi-Monthly Report with Billing Count and without Invoice Number

In [ ]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count

In [ ]:
month = np.array(['Mar'])
semi_month = np.array([])
for m in month:
    semi_month = np.append(semi_month, m + '_Mid')
    semi_month = np.append(semi_month, m + '_End')

bp = np.where(df_count.columns.values == 'Billing Period Start')

df_count.columns.values[bp] = semi_month

df_count

In [ ]:
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

In [ ]:
df_count.to_excel("03_2022 Semi-Monthly Report Others Count.xlsx", index = False, header = True)

### Export Semi-Monthly Report As Excel File

In [ ]:
# month = np.array(['Jan'])

# semi_month = np.array([])
# for m in month:
#     semi_month = np.append(semi_month, m + '_Mid')
#     semi_month = np.append(semi_month, m + '_End')

bp1 = np.where(df_output.columns.values == 'Billing Period Start')
dn1 = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in semi_month]

df_output.columns.values[bp1] = semi_month
df_output.columns.values[dn1] = dn_month

In [ ]:
df_output.groupby(['Accounting_CustomerID']).count().sort_values(['CustomerID'], ascending=False)
df_output

In [ ]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

In [ ]:
df_output.to_excel("03_2022 Semi-Monthly Report Others.xlsx", index = False, header = True)

In [ ]:
# bp = np.where(df_count.columns.values == 'Billing Period Start')
# df_count.columns.values[bp] = semi_month

In [ ]:
# writer2 = pd.ExcelWriter('2022 Semi-Monthly Report Others.xlsx', engine = 'xlsxwriter')
# df_count.to_excel(writer2, index = False, header = True, sheet_name = 'Summary')
# df_output.to_excel(writer2, index = False, header = True, sheet_name = 'Detail 01-02')
# writer2.save()

# MOVE FILES TO OUTPUT!

### Get Accessorial Items

In [ ]:
df_list = []

df_combine = pd.DataFrame()

month = ['Mar', 'Mar']

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail = df_detail[['CustomerID', 'DocNo', 'WISE Facility', 'Header Billing Period Start', 'Bill Pay Category',
                                 'BillingCode', 'Qty', 'Cost']]
    
    df_select_detail = df_select_detail.loc[df_select_detail['Bill Pay Category'] == 'ACCESSORIAL']
    
    
    df_select_detail['BillingCode'] = df_select_detail['BillingCode'].str.lower()
    
    df_select_detail['Item'] = np.nan
    
     
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'pallet[^4]|pallet\s[^4]', 
                                                                      regex=True, na=False), 'Item'] = 'Customized Size Pallet'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('chep', na=False), 'Item'] = 'CHEP Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('peco', na=False), 'Item'] = 'PECO Pallet'   
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('48x40', na=False), 'Item'] = 'Grade B Pallet'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('air bag', na=False), 'Item'] = 'Air Bag'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('brace', na=False), 'Item'] = 'Block And Brace'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('bol', na=False), 'Item'] = 'BOL'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('cancel', na=False), 'Item'] = 'Cancellation'
        
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('load bar', na=False), 'Item'] = 'Load Bar'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('appointment', na=False), 'Item'] = 'Missed Appointment'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('placard', na=False), 'Item'] = 'Placard'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('strap', na=False), 'Item'] = 'Plastic Strap'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('rush', na=False), 'Item'] = 'Rush Order'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('serial number', na=False), 'Item'] = 'Serial Number Tracking' 
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('sheet', na=False), 'Item'] = 'Slip Sheet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('wrap', na=False), 'Item'] = 'Stretch Wrap'

    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('box', na=False), 'Item'] = 'Brown Box'   
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('label', na=False), 'Item'] = 'Label'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('labor', na=False), 'Item'] = 'Labor'  
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('palletizing and', na=False), 
                         'Item'] = 'Palletizing And Wrapping'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*document)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*photo)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains(r'(?=.*picture)', 
                                                                     regex=True, na=False), 'Item'] = 'Picture/Photo/Document'
    
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('grade a', na=False), 'Item'] = 'Grade A Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('grade b', na=False), 'Item'] = 'Grade B Pallet'
    df_select_detail.loc[df_select_detail['BillingCode'].str.contains('corner', na=False), 'Item'] = 'Corner Board'
        
            
    df_select_detail['Item'] = df_select_detail['Item'].fillna('Others')
    
    
    df_sum_detail = df_select_detail.groupby(['CustomerID', 'DocNo', 'WISE Facility', 
                                                'Header Billing Period Start', 'Bill Pay Category', 
                                                'BillingCode', 'Item'])['Qty', 'Cost'].sum().reset_index()
    
        
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
    

    df_combine_header_detail = pd.merge(df_sum_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 'Billing Period Start', 
                                                            'DocNo', 'BillingCode', 'Item', 'Qty', 'Cost'])
    
    df_combine_month['Billing Period Start'] = month[excel]    
    
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)
    
    
    df_combine = df_combine.append(df_combine_month)
    
    print(df_combine.shape)

In [ ]:
df_list[len(df_list) - 1].sort_values('CustomerID')

In [ ]:
df_combine = df_combine.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_combine['WISE Facility'] = df_combine['WISE Facility'].str.title()
df_combine['Bill To'] = df_combine['Bill To'].str.upper()
df_combine

In [ ]:
df_combine.to_excel("03_2022 Accessorial Items Cost.xlsx", index = False, header = True)

# MOVE FILES TO OUTPUT!